In [5]:
import ast
import csv
from distutils.log import error 
import json
import math
import numpy as np

import led_config_utils
import mesh_config
from funky_lights import wavefront, led_config

from importlib import reload
reload(mesh_config)

START_OFFSET = 0.0
END_OFFSET = 0.0

LED_ID_MAPPING_RIGHT = np.array([
    [-1,  -1,  -1,   4,   3,   2,   1,   0,  -1,  -1,  -1],
    [-1,  -1,   5,   6,   7,   8,   9,  10,  11,  -1,  -1],
    [-1,  20,  19,  18,  17,  16,  15,  14,  13,  12,  -1],
    [21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31],
    [42,  41,  40,  39,  38,  37,  36,  35,  34,  33,  32],
    [43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53],
    [64,  63,  62,  61,  60,  59,  58,  57,  56,  55,  54],
    [65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75],
    [-1,  84,  83,  82,  81,  80,  79,  78,  77,  76,  -1],
    [-1,  -1,  85,  86,  87,  88,  89,  90,  91,  -1,  -1],
    [-1,  -1,  -1,  96,  95,  94,  93,  92,  -1,  -1,  -1]
])
LED_ID_MAPPING_RIGHT = np.rot90(LED_ID_MAPPING_RIGHT, k=2)

LED_ID_MAPPING_LEFT = np.array([
    [-1,  -1,  -1,  59,  58,  37,  36,   9,  -1,  -1,  -1],
    [-1,  -1,  78,  60,  57,  38,  35,   8,  10,  -1,  -1],
    [-1,  79,  77,  61,  56,  39,  34,   7,  11,   0,  -1],
    [90,  80,  76,  62,  55,  40,  33,   6,  12,   1,  -1],
    [89,  81,  75,  63,  54,  41,  32,   5,  13,   2,  -1],
    [88,  82,  74,  64,  53,  42,  31,   4,  14,  -1,  -1],
    [87,  83,  73,  65,  52,  43,  30,   3,  15,  21,  -1],
    [86,  84,  72,  66,  51,  44,  29,  22,  16,  20,  -1],
    [-1,  85,  71,  67,  50,  45,  28,  23,  17,  19,  -1],
    [-1,  -1,  70,  68,  49,  46,  27,  24,  18,  -1,  -1],
    [-1,  -1,  -1,  69,  48,  47,  26,  25,  -1,  -1,  -1]
])
LED_ID_MAPPING_LEFT = np.rot90(LED_ID_MAPPING_LEFT, k=2)


def Ry(theta):
  return np.matrix([[ math.cos(theta), 0, math.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-math.sin(theta), 0, math.cos(theta)]])

def createLEDPointsForEyes(name, mesh, points):
    for group in mesh.groups:
        if group.name == name:
            break

    group = next((g for g in mesh.groups if g.name == name), None)
    if not group:
        print("Couldn't find mesh group for %s" % name)
        return

    data = led_config_utils.prepare_data(group)
    _, U, mu = led_config_utils.pca(data)

    # Select mapping
    if name == 'eye_right':
        led_id_mapping = LED_ID_MAPPING_RIGHT
    elif name == 'eye_left':
        led_id_mapping = LED_ID_MAPPING_LEFT
    else:
        error("Should never get here.")

    points_2d = np.dot(points - mu, U.T)
    grid_size = 11
    center = points_2d.mean(0)
    radius = np.linalg.norm(points_2d - center, axis=1).mean()
    step_size = (radius * 2) / grid_size
    offset = math.floor(grid_size / 2) * step_size
    x0 = center[0] - offset
    y0 = center[1] - offset

    segment_points = []
    segment_ids = []
    for r in range(grid_size):
        row = []
        y_pos = y0 + r * step_size
        for c in range(grid_size):
            x_pos = x0 + c * step_size 
            id = led_id_mapping[grid_size - 1 - r][c]
            if id >= 0:
                segment_ids.append(id)
                segment_points.append(np.array([x_pos, y_pos, 0]))
                # print("%d %d %d" % (id, r, c))

    zipped_pairs = zip(segment_ids, segment_points)
    points = np.array([x for _, x in sorted(zipped_pairs)])
    # Transform points back into 3D coordinates
    points = np.dot(points, U) + mu
    
    return points


def createNodesFromCSV(csv_points):
    nodes = None
    prev_node = None
    # Rotate points around Y to match the Funky model orientation
    R = Ry(math.radians(90))
    for point in csv_points:
        point = np.array(point)
        point = (R * point.reshape((3,1))).reshape((1,3))
        point = np.squeeze(np.asarray(point))
        node = led_config_utils.Node(p=point)
        if nodes == None:
            nodes = node
        if prev_node:
            prev_node.next = node
        prev_node = node
    return nodes



all_segments = {}
# mesh = wavefront.load_obj(config.mesh)
with open('../config/funklet/funklet_actual.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    segments=[]

    for row in reader:
        uid = int(row[0])
        name = row[1]
        actual_num_leds = int(row[2])
        actual_length = float(row[3])
        reverse = (row[4] == 'TRUE')
        led_offset = int(row[5])
        sub_component = row[6]
        actual_num_adressable_leds = int(row[7])
        csv_points = ast.literal_eval(row[10])

        nodes = createNodesFromCSV(csv_points)
        if nodes == None:
            print('No nodes available.')
            continue
    
        modelled_length = led_config_utils.line_segments_length(nodes)
        modelled_length = modelled_length -  START_OFFSET -  END_OFFSET
        
        leds_distance = modelled_length / actual_num_adressable_leds
        points = led_config_utils.trace_line_segments(nodes, actual_num_adressable_leds, START_OFFSET, leds_distance)
        
        # if name == 'eye_right' or name == 'eye_left':
        #     points = createLEDPointsForEyes(name, mesh, points)
        
        if len(points) == 0:
            print('No points available.')
            continue

        if reverse:
            points = np.flip(points, axis=0)

        if led_offset > 0:
            points = np.concatenate((points[led_offset:], points[:led_offset]), axis=0)

        segment = led_config.Segment(
            uid=uid, name=name, points=points, num_leds=points.shape[0], length=actual_length)
        print('Segment %s: length=%.1fm, num_leds=%s' % (segment.name, segment.length, segment.num_leds))
        segments.append(segment)
        all_segments[uid] = segment

# Collapse some segments
# SEGMENT_1 = [122,1221,1222,1223,1224,1225]
# SEGMENT_2 = [2,21,22,23,24,25]
# SEGMENT_3 = [123,31,32,33,34,35,36,37,38,39]
# SEGMENT_4 = [4,41,42,43]


# for merge_list in [SEGMENT_1, SEGMENT_2, SEGMENT_3, SEGMENT_4]:
#     merged_segment = all_segments[merge_list[0]]
#     for uid in merge_list[1:]:
#         segment = all_segments.pop(uid)
#         merged_segment.merge(segment)

# Create LED config
config = led_config.LedConfig()
for segment in all_segments.values():
    config.led_segments.append(segment)
    config.total_num_segments += 1
    config.total_length += segment.length
    config.total_num_leds += segment.num_leds

with open('../config/led_config.json', 'w', encoding='utf-8') as f:
    json.dump(config.to_dict(), f, ensure_ascii=False, indent=4)



Segment ear_right: length=3.4m, num_leds=68
Segment head: length=3.3m, num_leds=65
Segment ear_left: length=3.4m, num_leds=68
Segment leg_back_left: length=4.5m, num_leds=88
Segment leg_back_right: length=4.0m, num_leds=79
Segment leg_front_left: length=4.2m, num_leds=82
Segment leg_front_right: length=3.8m, num_leds=74
Segment trunk: length=6.4m, num_leds=127
Segment tail: length=2.7m, num_leds=54
